<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Import_CV2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 07/25/2021-1
from __future__ import absolute_import
from IPython.display import clear_output

import glob, os, shutil
from os.path import *

try: 
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except Exception as err:
    pass

try: 
    from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt

try: 
    import cv2
    from cv2 import COLOR_BGR2RGBA
except Exception as err:
    print(err)
    %pip install opencv-python-headless # works.
    # %pip install opencv-contrib-python-headless
    import cv2
    from cv2 import COLOR_BGR2RGBA, IMREAD_UNCHANGED

import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

if exists(resizedImagesPath):
    pass
    shutil.rmtree(resizedImagesPath)

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    gdriveResizedImagesPth = join(mydrivePth, 'TensorflowResizedImages')
    
    os.chdir(mydrivePth)
    from TarfileFunctions import *
    from BashColors import C
    
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveResizedImagesPth, tensorflowImagesPath)
        # tff.listTarfiles()
        os.chdir(contentPth)
        # tff.extractTarfiles('Juno_Original_Images.tar.gz')

    gdrive_resized_images = '/content/gdrive/MyDrive/TensorflowResizedImages'
    if not exists(resizedImagesPath): # and not exists(resizedImagesPath):
        # print('from:', gdrive_resized_images)
        # print('to:', resizedImagesPath)
        try: shutil.copytree(gdrive_resized_images, resizedImagesPath)
        except Exception as err:
            print(err)

    os.chdir(contentPth)

except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C
    tarfilePath=join(contentPth, 'Juno_Original_Images.tar.gz')
    if exists(tarfilePath) and not exists(tensorflowImagesPath):
        tff.extractTarfiles('Juno_Original_Images.tar.gz')

rootPth = expanduser("~")

os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}\n')
junkList = glob.glob('*.png')
for path in junkList:
    fullPath = abspath(path)
    if basename(fullPath).__contains__('zoom'):
        print(fullPath)
        os.remove(fullPath)

In [ ]:
os.chdir(contentPth)
print(f'from: {C.Green}{os.getcwd()}{C.ColorOff}')
directoryList=[]
fileList=[]
dirCount=0
fileCount=0
globList = glob.glob('**', recursive=True)
for fil in sorted(globList):
    fullPath = abspath(fil)
    if isdir(fullPath):
        dirCount+=1
        print(f'\n{C.IBlue}{fullPath}{C.ColorOff}')
        directoryList.append(fullPath)
    elif isfile(fullPath) and fullPath.endswith('.png'):
        # fileCount+=1
        print(fileCount, fullPath)
        fileList.append(fullPath)
        fileCount+=1
# clear_output()

In [ ]:
#  Create backgroundImg.png
backgroundImg = np.zeros(shape=(224,224,4))
backgroundImgSavePath = join('/content/gdrive/MyDrive/TensorflowResizedImages',
                             'backgroundImage224.png')
backgroundImgSavePath = join(contentPth, 'backgroundImage224.png')
if not exists(backgroundImgSavePath):
    bgImg = cv2.imwrite(backgroundImgSavePath, backgroundImg)
bgImg = cv2.imread(backgroundImgSavePath, cv2.IMREAD_UNCHANGED)

cv2.waitKey(1)
cv2.destroyAllWindows()

bgZeroPixel = bgImg[0][0]
print( basename(backgroundImgSavePath))
print(f'size:{bgImg.size} shape: {bgImg.shape} ndim: {bgImg.ndim}')
print(f'bgZeroPixel: {bgZeroPixel}')
# clear_output()

In [ ]:
def changeBackgroundColors(path, image=None):
    splitPath = split(path)
    filePath = splitPath[0]
    fileName = splitPath[1]
    fileName = 'new_' + fileName
    print(filePath)
    print(fileName)
    saveImagePath = join(contentPth, fileName)

    print(f'saveImagePath: {saveImagePath}')
    original_image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    newImg = np.copy(original_image)

    zeroPixel = newImg[0][0]
    print('zeroPixel:', zeroPixel)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    pixelcount = 0

    width, height, channels = original_image.shape
    # print(width, height, channels)

    for x in range(0, width):
        for y in range(0, height):
            channels_xy = newImg[y][x]
            # print(channels_xy)
            if all(channels_xy == zeroPixel):
                pixelcount+=1
                newImg[y][x] == [64,64,64,255]
                
    # print(pixelcount)

    
    cv2_imshow(original_image)
    cv2.imwrite(saveImagePath, newImg)
    newImg = cv2.imread(saveImagePath)
    cv2_imshow(newImg)

changeBackgroundColors(path=backgroundImgSavePath)
clear_output()

In [ ]:
os.chdir(tensorflowImagesPath)
imageGlob= glob.glob('**', recursive=True)
imageGlobPathList=[]
count = 0
for path in sorted(imageGlob):
    path=abspath(path)
    if path.endswith('.png'):
        imageGlobPathList.append(path)
        count += 1
        print(f'{count}. {path}')
        # print(path)
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        zeroPixel = img[0][0]
        cv2_imshow(img)
        cv2.waitKey(500)
        print(f'size: {img.size} shape:{img.shape} ndim:{img.ndim}')
        print(f'zeroPixel: {zeroPixel}\n')
        cv2.destroyAllWindows()

imageGlobPathList=sorted(imageGlobPathList)


In [ ]:

def getCV2BackgroundImg(newShape=(224,224,4), bgColor=[0,0,0,0]):
    print(newShape)
    print(bgColor)
    new_image = np.zeros(shape=newShape, dtype=np.uint8)
    new_image.fill(0)


    savePath = join(contentPth, 'new_blank.png')
    os.remove(savePath)
    cv2.imwrite(savePath, new_image)
    print(new_image[0][0])
    cv2.waitKey(10)
    cv2.destroyAllWindows()
    return savePath

pth = getCV2BackgroundImg(bgColor=[0,0,0,0])
img=cv2.imread(pth)
cv2_imshow(img)
files.view(os.getcwd())

In [ ]:
print(len(imageGlobPathList), 'files')
count=0
for path in sorted(imageGlobPathList):
    count+=1
    imagePath = abspath(path)
    print(imagePath)
    imageName = basename(imagePath)
    
    img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    # print(type(img))
    # print('original image:',img.shape)
    
    zoomImg_H = cv2.hconcat([bgImg, img, bgImg])
    bgImg_H = cv2.hconcat([bgImg, bgImg, bgImg])
    
    newImg = cv2.vconcat([bgImg_H, zoomImg_H, bgImg_H])
    newImg = cv2.resize(newImg, (224, 224), cv2.INTER_CUBIC)
    
    imageName = 'zoom_' + imageName
    saveImagePath = join(contentPth, imageName)
    # print('saveImagePath:', saveImagePath)
    
    if not exists(saveImagePath):
        cv2.imwrite(saveImagePath, newImg)
    newImg = cv2.imread(saveImagePath, cv2.IMREAD_UNCHANGED)

    print(f'\n{count}. {imageName}')
    cv2_imshow(newImg)
    cv2.waitKey(1500)
    cv2.destroyAllWindows()
    print('ndim:', newImg.ndim, 'shape:', newImg.shape, 'size:', newImg.size)

In [ ]:

cv2_imshow(img)
# plt.imshow(img)
bgPixel = img[1][1]
print(bgPixel.shape)
count1=0
count2=0
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        thisPixel = img[i][j]

        if thisPixel.all() == bgPixel.all():
            count1+=1

        if thisPixel[0] == 64:
            thisPixel[0] = 0

        elif thisPixel[1] == 64:
            count1+=1
            thisPixel[1] = 0

        elif thisPixel[2] == 64:
            thisPixel[2] = 0

        if thisPixel.all() == bgPixel.all():
            count2+=1
            # print(a[3])
        else: pass

        #print(img[i][j])
        
print('count1:', count1)
print('count2:', count2)
print('img.size:', img.size)
print(thisPixel.shape)

In [ ]:

b,g,r,a = cv2.split(img)
print(b,g,r,a)

newImg = cv2.merge([b,g,r,a])
plt.imshow(newImg)

newImgPath = join(contentPth, 'newEnemy.png')
cv2.imwrite(filename=newImgPath, img=newImg)

try: 
    gdriveNewImgPath = join(mydrivePth, 'newEnemy.png')
    cv2.imwrite(gdriveNewImgPath, newImg)
except: pass



In [ ]:

imgZeroPixel=img[0][0]
bgCount=0
pxCount=0
print(f'bgPixel: {imgZeroPixel}')
plt.imshow(img)

In [ ]:
# Read the image
imagePath=join(contentPth, "zoom_enemy_1.png")

original_image = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
# Not necessary. Make a copy to plot later
new_img = np.copy(original_image)
cv2.waitKey(500)
cv2.destroyAllWindows()

# Plot the images
fig=plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(original_image)
ax1.set_title('Original Image')

ax2 = fig.add_subplot(1,2,2)
ax2.imshow(new_img)
ax2.set_title('New Image')
plt.show()

In [ ]:
print(imagePath)

img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
cv2_imshow(img)
cv2.waitKey(1)
cv2.destroyAllWindows()

height, width, channels = img.shape
print(img.shape)
print(img[0][0])
white = [0,0,0,0]
black = [255,255,255,255]
blackground = [64,64,64,255]

for x in range(0,width):
    for y in range(0,height):
        channels_xy = img[y,x]
        if all(channels_xy == white):    
            img[y,x] = blackground

        # elif all(channels_xy == black):
            # img[y,x] = white

cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()